# A3
* GCR = 0.4
* Albedo = 0.2
* Hub Height = 3.5
* Configuration = 1-Up portrait
* Ground surface = Horizontal
***
* A GCR of 0.4 for the Trina Vertex module (length = 2.384m) corresponds to a rowto-row spacing or pitch of 5.96m
* 1-up Portrait: 5 rows, each with 25 modules
* Torque tube diameter = 15 cm. (round)
* Maximum tracker rotation angle = 55 deg
* Backtracking is enabled
* Location: Albuquerque, New Mexico USA (35.05°, -106.54°)

In [1]:
import os
from pathlib import Path

testfolder = 'Scenarios/A3-1'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in Scenarios/A3-1


In [2]:
import bifacial_radiance as br
import bifacialvf as bf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, platform
import csv
from pathlib import Path

In [3]:
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)
print("pyplot ", plt.matplotlib.__version__)
print("bifacialvf version ", bf.__version__)

Working on a  Darwin 23.3.0
Python version  3.11.5 (main, Sep 11 2023, 08:19:27) [Clang 14.0.6 ]
Pandas version  2.0.3
bifacial_radiance version  0.4.2+246.gf6f42f5.dirty
pyplot  3.7.2
bifacialvf version  0.1.8.1


In [4]:
simulationname = 'A3-1'

# Location Albuquerque, New Mexico, USA
lat = 35.05
lon = -106.54

# Scene Parameters
azimuth=90
tilt=30

# MakeModule Parameters
moduletype='PVmod'
numpanels=1
module_x = 1.303 # m
module_y = 2.384 # m. slope we will measure
sensorsy=2
torquetube_diam = 0.15

zgap = 0.02 # m
xgap = 0.002 # m

# SceneDict Parameters
pitch = 5.96 # m
albedo = 0.2
hub_height = 3.5 # m  
nMods = 25 
nRows = 5

sceneDict = {'tilt':tilt,'pitch':pitch,'albedo': albedo,'hub_height':hub_height,'azimuth':azimuth, 'nMods': nMods, 'nRows': nRows} 

In [5]:
demo = br.RadianceObj(simulationname,path = testfolder)
demo.setGround(albedo)

path = Scenarios/A3-1
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [6]:
cumulativesky = False
trackerParams = {
    'limit_angle': 55,
    'backtrack': True,
    'gcr': 0.4,
    'azimuth': 180,  # axis angle, N-S = 180
    'cumulativesky': cumulativesky
}

In [7]:
module=demo.makeModule(name=moduletype,x=module_x,y=module_y, xgap=xgap, zgap=zgap)


Module Name: PVmod
Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [8]:
module.addTorquetube(diameter=torquetube_diam, tubetype='Round', material='Metal_Grey', 
                     axisofrotation=True, visible=True, recompile=True)

Module PVmod updated in module.json
Pre-existing .rad file objects/PVmod.rad will be overwritten



In [9]:
weatherfile = str(Path().resolve().parent.parent / 'Instructions' / 'Phase2_meteo_hourly_psm3format.csv')
weatherfile

'/Users/storopov/Documents/GitHub/bifacial_radiance/docs/development/Instructions/Phase2_meteo_hourly_psm3format.csv'

In [10]:
metdata = demo.readWeatherFile(weatherfile, source='sam', starttime='2022-03-22_0000', endtime='2022-03-23_0000')

COLUMN DATAS Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DNI', 'DHI',
       'temp_air', 'wind_speed', 'humidity', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2022
Filtering dates
Saving file EPWs/metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is left-labeled  with a delta of +30 mins. i.e. 12 is 12:30 sunpos.


In [11]:
trackerdict = demo.set1axis(**trackerParams)

In [12]:
demo.gendaylit1axis()

Creating ~11 skyfiles. 
Created 11 skyfiles in /skies/


{'2022-03-22_0730': {'surf_azm': 90.0,
  'surf_tilt': 53.28,
  'theta': -53.28,
  'dni': 0,
  'ghi': 65,
  'dhi': 62,
  'temp_air': 2.4,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0730.rad'},
 '2022-03-22_0830': {'surf_azm': 90.0,
  'surf_tilt': 53.44,
  'theta': -53.44,
  'dni': 2,
  'ghi': 182,
  'dhi': 180,
  'temp_air': 3.0,
  'wind_speed': 1.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0830.rad'},
 '2022-03-22_0930': {'surf_azm': 90.0,
  'surf_tilt': 38.35,
  'theta': -38.35,
  'dni': 3,
  'ghi': 279,
  'dhi': 276,
  'temp_air': 3.9,
  'wind_speed': 2.0,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_0930.rad'},
 '2022-03-22_1030': {'surf_azm': 90.0,
  'surf_tilt': 21.72,
  'theta': -21.72,
  'dni': 172,
  'ghi': 504,
  'dhi': 361,
  'temp_air': 5.0,
  'wind_speed': 3.1,
  'skyfile': 'skies/sky2_35.05_-106.54_2022-03-22_1030.rad'},
 '2022-03-22_1130': {'surf_azm': 90.0,
  'surf_tilt': 3.91,
  'theta': -3.91,
  'dni': 13,
  'ghi': 291,
  'dhi'

In [13]:
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict)
trackerdict = demo.makeOct1axis()


Making ~11 .rad files for gendaylit 1-axis workflow (this takes a minute..)
11 Radfiles created in /objects/

Making 11 octfiles in root directory.
Created 1axis_2022-03-22_0730.oct
Created 1axis_2022-03-22_0830.oct
Created 1axis_2022-03-22_0930.oct
Created 1axis_2022-03-22_1030.oct
Created 1axis_2022-03-22_1130.oct
Created 1axis_2022-03-22_1230.oct
Created 1axis_2022-03-22_1330.oct
Created 1axis_2022-03-22_1430.oct
Created 1axis_2022-03-22_1530.oct
Created 1axis_2022-03-22_1630.oct
Created 1axis_2022-03-22_1730.oct


In [14]:
trackerdict = demo.analysis1axis(sensorsy=2) # Middle Module

Linescan in process: 1axis_2022-03-22_0730_Row3_Module13_Front
Linescan in process: 1axis_2022-03-22_0730_Row3_Module13_Back
Saved: results/irr_1axis_2022-03-22_0730_Row3_Module13.csv
Index: 2022-03-22_0730. Wm2Front: 46.071178333333336. Wm2Back: 12.507541666666665
Linescan in process: 1axis_2022-03-22_0830_Row3_Module13_Front
Linescan in process: 1axis_2022-03-22_0830_Row3_Module13_Back
Saved: results/irr_1axis_2022-03-22_0830_Row3_Module13.csv
Index: 2022-03-22_0830. Wm2Front: 143.6915666666667. Wm2Back: 34.45149333333333
Linescan in process: 1axis_2022-03-22_0930_Row3_Module13_Front
Linescan in process: 1axis_2022-03-22_0930_Row3_Module13_Back
Saved: results/irr_1axis_2022-03-22_0930_Row3_Module13.csv
Index: 2022-03-22_0930. Wm2Front: 249.34648333333334. Wm2Back: 40.23852666666667
Linescan in process: 1axis_2022-03-22_1030_Row3_Module13_Front
Linescan in process: 1axis_2022-03-22_1030_Row3_Module13_Back
Saved: results/irr_1axis_2022-03-22_1030_Row3_Module13.csv
Index: 2022-03-22_103

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A3-Mar-22-M.csv')

In [15]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 1) # South Module

Linescan in process: 1axis_2022-03-22_0730_Row3_Module1_Front
Linescan in process: 1axis_2022-03-22_0730_Row3_Module1_Back
Saved: results/irr_1axis_2022-03-22_0730_Row3_Module1.csv
Index: 2022-03-22_0730. Wm2Front: 47.98189833333333. Wm2Back: 14.998038333333334
Linescan in process: 1axis_2022-03-22_0830_Row3_Module1_Front
Linescan in process: 1axis_2022-03-22_0830_Row3_Module1_Back
Saved: results/irr_1axis_2022-03-22_0830_Row3_Module1.csv
Index: 2022-03-22_0830. Wm2Front: 149.76136666666667. Wm2Back: 41.77771333333334
Linescan in process: 1axis_2022-03-22_0930_Row3_Module1_Front
Linescan in process: 1axis_2022-03-22_0930_Row3_Module1_Back
Saved: results/irr_1axis_2022-03-22_0930_Row3_Module1.csv
Index: 2022-03-22_0930. Wm2Front: 254.9287333333333. Wm2Back: 50.883354999999995
Linescan in process: 1axis_2022-03-22_1030_Row3_Module1_Front
Linescan in process: 1axis_2022-03-22_1030_Row3_Module1_Back
Saved: results/irr_1axis_2022-03-22_1030_Row3_Module1.csv
Index: 2022-03-22_1030. Wm2Front:

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A3-Mar-22-S.csv')

In [16]:
trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module

Linescan in process: 1axis_2022-03-22_0730_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0730_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0730_Row3_Module25.csv
Index: 2022-03-22_0730. Wm2Front: 47.929700000000004. Wm2Back: 14.958321666666665
Linescan in process: 1axis_2022-03-22_0830_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0830_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0830_Row3_Module25.csv
Index: 2022-03-22_0830. Wm2Front: 149.25025. Wm2Back: 41.09414833333334
Linescan in process: 1axis_2022-03-22_0930_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_0930_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_0930_Row3_Module25.csv
Index: 2022-03-22_0930. Wm2Front: 253.8906. Wm2Back: 49.84613666666667
Linescan in process: 1axis_2022-03-22_1030_Row3_Module25_Front
Linescan in process: 1axis_2022-03-22_1030_Row3_Module25_Back
Saved: results/irr_1axis_2022-03-22_1030_Row3_Module25.csv
Index: 2022-03-22_1030. Wm2Front: 512.7

Traceback (most recent call last):
  File "/Users/storopov/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/nl/yn7kzq_959v4881bh526ml9mpgtmyb/T/ipykernel_99268/984878894.py", line 1, in <module>
    trackerdict = demo.analysis1axis(sensorsy=2, modWanted = 25) # North Module
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py", line 2884, in analysis1axis
    analysis.analysis(octfile=octfile,name=name,frontscan=frontscanind,backscan=backscanind,accuracy=accuracy)
  File "/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/main.py", line 5153, in analysis
    backDict = self._irrPlot(octfile, linepts, name+'_Back',
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/storopov/Documents/GitHub/bifacial_radiance/bifacial_radiance/ma

In [ ]:
demo.calculateResults()

In [ ]:
demo.CompiledResults.set_index('timestamp', inplace=True)
demo.CompiledResults
# Grear_mean and Gfront_mean values to be recorded in the excel

In [ ]:
demo.exportScenarioData(trackerdict, savefile = Path().resolve().parent.parent / 'Results' / 'A3-Mar-22-N.csv')